#Майнор ИАД. Домашнее задание 3. YOLO.

В этом задании вы напишете и обучите свой собственный YOLO детектор. Нужно будет разобраться со статьей: понять какого формата должна быть обучающая пара (x, y), как перевести лосс из математической формулы в питоновский код - ну и конечно понять и реализовать саму архитектуру модели.

Выборка на котрой мы будем обучать модель состоит из разнообразных фотографий яблок, бананов и апельсинов. Данные скачиваем [отсюда](https://drive.google.com/file/d/1d8GSfZoWbraWCSUhX78yl4CnMFYE-5n3/view?usp=sharing).

Баллы за ДЗ распределены следующим образом: 
- Выборка для YoloV1 - 2 балла
- YOLO модель - 2 балла
- YOLO Loss - 3 балла
- Вспомогательные функции - 2 балла
- Обучение и расчет метрик - 2 балла

Для построения и обучения можно использовать как pytorch, так и pytorch-lightning.

In [ ]:
# Данная библиотека понадобится нам, чтобы обработать разметку
! pip install xmltodict pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 800 kB 14.9 MB/s 
     |████████████████████████████████| 512 kB 75.6 MB/s 
     |████████████████████████████████| 125 kB 70.7 MB/s 


Скачаем данные

In [ ]:
!wget --quiet --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://drive.google.com/uc?export=download&id=1d8GSfZoWbraWCSUhX78yl4CnMFYE-5n3' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1d8GSfZoWbraWCSUhX78yl4CnMFYE-5n3" -O data.zip && rm -rf /tmp/cookies.txt
!unzip -q data.zip
!rm data.zip
!ls -l

total 12
drwx------ 4 root root 4096 Nov 26 08:32 data
drwxr-xr-x 3 root root 4096 Dec 19 18:25 __MACOSX
drwxr-xr-x 1 root root 4096 Dec 16 21:15 sample_data


In [ ]:
!cat data/train/apple_3.xml

<annotation>
	<folder>train</folder>
	<filename>apple_3.jpg</filename>
	<path>C:\tensorflow1\models\research\object_detection\images\train\apple_3.jpg</path>
	<source>
		<database>Unknown</database>
	</source>
	<size>
		<width>1000</width>
		<height>708</height>
		<depth>3</depth>
	</size>
	<segmented>0</segmented>
	<object>
		<name>apple</name>
		<pose>Unspecified</pose>
		<truncated>1</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>584</xmin>
			<ymin>438</ymin>
			<xmax>867</xmax>
			<ymax>708</ymax>
		</bndbox>
	</object>
	<object>
		<name>apple</name>
		<pose>Unspecified</pose>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>492</xmin>
			<ymin>141</ymin>
			<xmax>740</xmax>
			<ymax>394</ymax>
		</bndbox>
	</object>
	<object>
		<name>apple</name>
		<pose>Unspecified</pose>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>176</xmin>
			<ymin>199</ymin>
			<xmax>490</xmax>
			<ymax>466</ymax>
		</bndbox>
	</object>
	<obj

## Релизуйте выборку для YoloV1 - 2 балла

In [ ]:
import os
import cv2
import json
import glob
import tqdm
import xmltodict
import math

from IPython.core.display import struct

from typing import List

import pandas as pd
import numpy as np

import torch
import torchvision
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl

import albumentations as A
import albumentations.pytorch

from PIL import Image

import matplotlib.pyplot as plt

from sklearn.metrics import auc
# Добавьте необходимые вам библиотеки, если их не окажется в списке выше

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
!pip3 install xml_to_dict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def intersection_over_union(predicted_bbox, gt_bbox) -> float:
    """
    Intersection Over Union для двух прямоугольников

    :param: predicted_bbox - [x_min, y_min, x_max, y_max]
    :param: gt_bbox - [x_min, y_min, x_max, y_max]
    
    :return: Intersection Over Union
    """

    intersection_bbox = np.array(
        [
            max(predicted_bbox[0], gt_bbox[0]),
            max(predicted_bbox[1], gt_bbox[1]),
            min(predicted_bbox[2], gt_bbox[2]),
            min(predicted_bbox[3], gt_bbox[3]),
        ]
    )

    intersection_area = max(intersection_bbox[2] - intersection_bbox[0], 0) * max(
        intersection_bbox[3] - intersection_bbox[1], 0
    )
    area_dt = (predicted_bbox[2] - predicted_bbox[0]) * (predicted_bbox[3] - predicted_bbox[1])
    area_gt = (gt_bbox[2] - gt_bbox[0]) * (gt_bbox[3] - gt_bbox[1])

    union_area = area_dt + area_gt - intersection_area

    iou = intersection_area / union_area
    return iou

In [ ]:
from xml_to_dict import XMLtoDict

In [ ]:
class2tag = {"apple": 1, "orange": 2, "banana": 3}

class FruitDataset(Dataset):
    def __init__(self, data_dir, transforms=None):
        self.image_paths = glob.glob(os.path.join(data_dir, "*.jpg")) ## YOUR CODE
        self.image_paths.sort()
        self.box_paths = (glob.glob(os.path.join(data_dir, "*.xml"))) ## YOUR CODE
        self.box_paths.sort() #https://github.com/hse-ds/iad-deep-learning/blob/master/2022/seminars/sem06/sem06_solved.ipynb
        print(self.box_paths)
        print(self.image_paths)
        assert len(self.image_paths) == len(self.box_paths)

        self.transforms = transforms

    # Координаты прямоугольников советуем вернуть именно в формате (x_center, y_center, width, height)
    def __getitem__(self, idx):

        image = np.array(Image.open(self.image_paths[idx]).convert("RGB")) #https://github.com/hse-ds/iad-deep-learning/blob/master/2022/seminars/sem06/sem06_solved.ipynb
        print(image.shape)
        im_w = image.shape[1]
        im_h = image.shape[0]
        boxes, class_labels = self.__get_boxes_from_xml(self.box_paths[idx], im_w, im_h)

        if self.transforms:
          transformed = self.transforms(image=image, bboxes=boxes, class_labels=class_labels)
          transformed_image = transformed['image']
          transformed_bboxes = transformed['bboxes']
          transformed_class_labels = transformed['class_labels'] #https://albumentations.ai/docs/getting_started/bounding_boxes_augmentation/

        target_tensor = np.zeros((7,7,13))

        for bbox_id in range(len(transformed_bboxes)):
          bbox = transformed_bboxes[bbox_id]
          for l in range(7):
            for c in range(7):
              if (bbox[0] >= l/7) and (bbox[0] <= 2*l/7) and (bbox[1] >= c/7) and (bbox[1] <= 2*c/7):
                if target_tensor[l][c][0] == 0:
                  target_tensor[l][c][0] = bbox[0] - (l/7) * 7
                  target_tensor[l][c][1] = bbox[1] - (c/7) * 7
                  target_tensor[l][c][2] = bbox[2]
                  target_tensor[l][c][3] = bbox[3]
                  target_tensor[l][c][4] = 1.0
                  cell = torch.tensor([l/7, c/7, (l + 1)/7, (c + 1)/7])
                  obj = torch.tensor([bbox[0] - (0.5 * bbox[2]), bbox[1] - (0.5 * bbox[3]), bbox[0] + (0.5 * bbox[2]), bbox[1] + (0.5 * bbox[3])])
                  confidence = intersection_over_union(cell, obj)
                  target_tensor[l][c][4] = confidence
                  
                  break
                else:
                  target_tensor[l][c][0] = bbox[0] - (l/7) * 7
                  target_tensor[l][c][1] = bbox[1] - (c/7) * 7
                  target_tensor[l][c][7] = bbox[2]
                  target_tensor[l][c][8] = bbox[3]
                  cell = torch.tensor([l/7, c/7, (l + 1)/7, (c + 1)/7])
                  obj = torch.tensor([bbox[0] - (0.5 * bbox[2]), bbox[1] - (0.5 * bbox[3]), bbox[0] + (0.5 * bbox[2]), bbox[1] + (0.5 * bbox[3])])
                  confidence = intersection_over_union(cell, obj)
                  target_tensor[l][c][4] = confidence

                  break
                target_tensor[l][c][9+class_labels[bbox_id]] = 1.0
                print('bbox1', bbox[0] - (l/7) * 7, bbox[1] - (col/7) * 7, l, c, bbox)
                
                break

        return torch.tensor(np.swapaxes(image, 0, 2), dtype=float)/255.0, torch.tensor(target_tensor, dtype=float)


    def __len__(self):
        return len(self.image_paths)


    def __get_boxes_from_xml(self, xml_filename: str, im_w, im_h):
      """
      Метод, который считает и распарсит (с помощью xmltodict) переданный xml
      файл и вернет координаты прямоугольников обьектов на соответсвующей фотографии
      и название класса обьекта в каждом прямоугольнике

      Обратите внимание, что обьектов может быть как несколько, так и один единственный
      """
      boxes = []
      class_labels = []

      parsed_xml = xmltodict.parse(open(xml_filename).read())

      if type(parsed_xml.get("annotation").get("object")) == list:
        for n in parsed_xml.get("annotation").get("object"):
          boxes.append(self.__convert_to_yolo_box_params([float(n.get('bndbox').get("xmin")), float(n.get('bndbox').get("ymin")), float(n.get('bndbox').get("xmax")), float(n.get('bndbox').get("xmax")), float(n.get("name"))], im_w, im_h))
          class_labels.append(n.get("name"))
      else:
        boxes.append(self.__convert_to_yolo_box_params([float(parsed_xml.get('annotation').get('object').get('bndbox').get('xmin')), float(parsed_xml.get('annotation').get('object').get('bndbox').get('ymin')), float(parsed_xml.get('annotation').get('object').get('bndbox').get('xmax')), float(parsed_xml.get('annotation').get('object').get('bndbox').get('ymax'))], im_w, im_h))
        class_labels.append(parsed_xml.get('annotation').get('object').get('bndbox').get('name'))

      print(boxes)  
      return boxes, class_labels

    def __convert_to_yolo_box_params(self, box_coordinates: List[int], im_w, im_h):
      """
      Перейти от [xmin, ymin, xmax, ymax] к [x_center, y_center, width, height].
      
      Обратите внимание, что параметры [x_center, y_center, width, height] - это
      относительные значение в отрезке [0, 1]

      :param: box_coordinates - координаты коробки в формате [xmin, ymin, xmax, ymax]
      :param: im_w - ширина исходного изображения
      :param: im_h - высота исходного изображения

      :return: координаты коробки в формате [x_center, y_center, width, height]
      """

      ans = []

      ans.append((box_coordinates[0] + box_coordinates[2]) / 2 / im_w)  # x_center
      ans.append((box_coordinates[1] + box_coordinates[3]) / 2 / im_h)  # y_center
      
      ans.append((box_coordinates[2] - box_coordinates[0]) / im_w)  # width
      ans.append((box_coordinates[3] - box_coordinates[1]) / im_h)  # height
      return ans

In [ ]:
WIDTH = 448
HEIGHT = 448

train_transform = A.Compose([A.Resize(height=HEIGHT, width=WIDTH),
                             A.pytorch.transforms.ToTensorV2()],
                             bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

test_transform = A.Compose([A.Resize(height=448, width=448),
                            A.pytorch.transforms.ToTensorV2()],
                            bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'])) #https://albumentations.ai/docs/getting_started/bounding_boxes_augmentation/

In [ ]:
train_dataset = FruitDataset(
    transforms=train_transform,
    data_dir="./data/train/"
    )

val_dataset = FruitDataset(
    transforms=test_transform, 
    data_dir="./data/test/"
    )


['./data/train/apple_1.xml', './data/train/apple_10.xml', './data/train/apple_11.xml', './data/train/apple_12.xml', './data/train/apple_13.xml', './data/train/apple_14.xml', './data/train/apple_15.xml', './data/train/apple_16.xml', './data/train/apple_17.xml', './data/train/apple_18.xml', './data/train/apple_19.xml', './data/train/apple_2.xml', './data/train/apple_20.xml', './data/train/apple_21.xml', './data/train/apple_22.xml', './data/train/apple_23.xml', './data/train/apple_24.xml', './data/train/apple_25.xml', './data/train/apple_26.xml', './data/train/apple_27.xml', './data/train/apple_28.xml', './data/train/apple_29.xml', './data/train/apple_3.xml', './data/train/apple_30.xml', './data/train/apple_31.xml', './data/train/apple_32.xml', './data/train/apple_33.xml', './data/train/apple_35.xml', './data/train/apple_36.xml', './data/train/apple_37.xml', './data/train/apple_38.xml', './data/train/apple_39.xml', './data/train/apple_4.xml', './data/train/apple_40.xml', './data/train/app

In [ ]:
# Немного проверок, чтобы убедиться в правильности направления решения
assert isinstance(train_dataset[0], tuple)
assert len(train_dataset[0]) == 2
assert isinstance(train_dataset[0][0], torch.Tensor)
print("Тесты успешно пройдены")

(349, 349, 3)
[[0.48567335243553006, 0.5214899713467048, 0.9255014326647565, 0.9570200573065902]]
(349, 349, 3)
[[0.48567335243553006, 0.5214899713467048, 0.9255014326647565, 0.9570200573065902]]
(349, 349, 3)
[[0.48567335243553006, 0.5214899713467048, 0.9255014326647565, 0.9570200573065902]]
Тесты успешно пройдены


In [ ]:
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size= 4,
    shuffle=True)

val_dataloader = DataLoader(
    dataset=val_dataset,
    batch_size=4, 
    shuffle=False
    )

Теперь начинается основная часть домашнего задания: обучите модель YOLO для object detection на __обучающем__ датасете. 

 - Создайте модель и функцию ошибки YoloV1 прочитав [оригинальную статью](https://paperswithcode.com/paper/you-only-look-once-unified-real-time-object)
 - Напишите функцию обучения модели
 - Используйте аугментации

## Реализуйте Модель - 2 балла

Our network has 24
convolutional layers followed by 2 fully connected layers.


In [ ]:
class CNNBlock(nn.Module):  # можно поменять на Lightning
    def __init__(self, in_channels, out_channels, is_max_pool:bool=False, **kwargs):
        
        super().__init__()

        self.conv = nn.Conv2d(in_channels, out_channels, **kwargs)
        self.batchnorm = nn.BatchNorm2d(out_channels, dtype=float)  # в статье еще не знали про батчнорм, но мы то из будущего ...
        self.leakyrelu = nn.LeakyReLU(0.1)

        self.is_maxpool = is_max_pool  # не после каждой свертки нужно делать maxpool
        self.maxpool = nn.MaxPool2d(2, stride=2)
        
    def forward(self, x):
        x = self.leakyrelu(self.batchnorm(self.conv(x)))

        if self.is_maxpool:
            x = self.maxpool(x)

        return x
    
class YOLO(nn.Module):
    def __init__(self, S=7, B=2, C=3):
        """
        :param: S * S - количество ячеек на которые разбивается изображение
        :param: B - количество предсказанных прямоугольников в каждой ячейке
        :param: C - количество классов
        """
        
        super(YOLO, self).__init__()

        self.S = S
        self.B = B
        self.C = C

        self.con1 = CNNBlock(in_channels=3, out_channels=64, kernel_size=(7,7), is_max_pool=True, stride=2, dtype=float)

        self.con2 = CNNBlock(in_channels=64, out_channels=192, kernel_size=(3,3), is_max_pool=True, dtype=float)

        self.con3 = nn.Sequential(
            CNNBlock(in_channels=192, out_channels=128, kernel_size=(1,1), is_max_pool=False, dtype=float),
            CNNBlock(in_channels=128, out_channels=256, kernel_size=(3,3), is_max_pool=False, dtype=float),
            CNNBlock(in_channels=256, out_channels=256, kernel_size=(1,1), is_max_pool=False, dtype=float),
            CNNBlock(in_channels=256, out_channels=512, kernel_size=(3,3), is_max_pool=True, dtype=float)
            )
        
        self.con4 = nn.Sequential(
            CNNBlock(in_channels=512, out_channels=256, kernel_size=(1,1), is_max_pool=False, dtype=torch.float64),
            CNNBlock(in_channels=256, out_channels=512, kernel_size=(3,3), is_max_pool=False, dtype=torch.float64),
            CNNBlock(in_channels=512, out_channels=256, kernel_size=(1,1), is_max_pool=False, dtype=torch.float64),
            CNNBlock(in_channels=256, out_channels=512, kernel_size=(3,3), is_max_pool=False, dtype=torch.float64),
            CNNBlock(in_channels=512, out_channels=256, kernel_size=(1,1), is_max_pool=False, dtype=torch.float64),
            CNNBlock(in_channels=256, out_channels=512, kernel_size=(3,3), is_max_pool=False, dtype=torch.float64),
            CNNBlock(in_channels=512, out_channels=512, kernel_size=(1,1), is_max_pool=False, dtype=torch.float64),
            CNNBlock(in_channels=512, out_channels=1024, kernel_size=(3,3), is_max_pool=True, dtype=torch.float64)
        )
        
        '''self.con5 = nn.Sequential(
            CNNBlock(in_channels=1024, out_channels=512, kernel_size=(1,1), dtype=float),
            CNNBlock(in_channels=512, out_channels=1024, kernel_size=(3,3), dtype=float),
            CNNBlock(in_channels=1024, out_channels=512, kernel_size=(1,1), dtype=float),
            CNNBlock(in_channels=512, out_channels=1024, kernel_size=(3,3), dtype=float),
            CNNBlock(in_channels=1024, out_channels=1024, kernel_size=(1,1), dtype=float),
            CNNBlock(in_channels=1024, out_channels=1024, kernel_size=(3,3), stride=2, dtype=float)
            )'''
        
        self.con5 = nn.Sequential(
            CNNBlock(in_channels=1024, out_channels=1024, kernel_size=(1,1), dtype=float),
            CNNBlock(in_channels=1024, out_channels=1024, kernel_size=(3,3), dtype=float)
        )

        self.con6 = nn.Sequential(
            CNNBlock(in_channels=1024, out_channels=1024, kernel_size=(1,1), dtype=float),
            CNNBlock(in_channels=1024, out_channels=1024, kernel_size=(1,1), dtype=float)
        )

        self.flatted = nn.Flatten()

        self.linear = nn.Sequential(
            nn.Linear(in_features=9216, out_features=2048, dtype=float),
            nn.LeakyReLU(0.1),
            nn.Dropout(),
            nn.Linear(in_features=2048, out_features=(7 * 7) * ((5 * 2) + 3), dtype=float)
        )

    def forward(self, x):
        x = self.con1(x)
        print('ok')
        x = self.con2(x)
        print('ok')
        x = self.con3(x)
        print('ok')
        x = self.con4(x)
        print('ok')
        x = self.con5(x)
        print('ok')
        x = self.con6(x)
        print('ok')
        x = self.flatted(x)
        print('ok')
        x = self.linear(x)
        print('ok')
        
        return x


In [ ]:
temp_model = YOLO()
#temp_model.to(device)
temp_model.eval()
expected_output_shape = temp_model.S * temp_model.S * (5 * temp_model.B + temp_model.C)

test_image = train_dataset[0][0]
test_image = test_image[None, :]

assert temp_model(test_image).reshape(-1).shape[0] == train_dataset[0][1].reshape(-1).shape[0]
assert temp_model(test_image).reshape(-1).shape[0] == expected_output_shape

(349, 349, 3)
[[0.48567335243553006, 0.5214899713467048, 0.9255014326647565, 0.9570200573065902]]
ok
ok
ok
ok
ok
ok
ok
ok
(349, 349, 3)
[[0.48567335243553006, 0.5214899713467048, 0.9255014326647565, 0.9570200573065902]]
ok
ok
ok
ok
ok
ok
ok
ok


## Реализуйте YoloLoss - 3 балла

In [ ]:
class YoloLoss(nn.Module):
    def __init__(self, S=7, B=2, C=3):
        """
        :param: S * S - количество ячеек на которые разбивается изображение
        :param: B - количество предсказанных прямоугольников в каждой ячейке
        :param: C - количество классов
        """
        
        super().__init__()
        self.mse = nn.MSELoss(reduction="sum")

        self.S = S
        self.B = B
        self.C = C

        self.lambda_noobj = 0.5
        self.lambda_coord = 5

    def intersection_over_union_small1(mlist, i, j, b = self.S):
      
      x = mlist[0]
      y = mlist[1]
      w = mlist[2]
      h = mlist[3]

      x_min = (i / b + mlist[0] / b) - mlist[2] / 2
      y_min = (j / b + mlist[1] / b) - mlist[3] / 2
      x_max = (i / b + mlist[0] / b) + mlist[2] / 2
      y_max = (j / b + mlist[1] / b) + mlist[3] / 2

      f = [x_min, y_min, x_max, y_max]

      return f

    def intersection_over_union_small2(mlist, i, j, b = self.S):
      
      x = mlist[0]
      y = mlist[1]
      w = mlist[2]
      h = mlist[3]

      x_min = (i / b + mlist[5] / b) - mlist[7] / 2
      y_min = (j / b + mlist[6] / b) - mlist[8] / 2
      x_max = (i / b + mlist[5] / b) + mlist[7] / 2
      y_max = (j / b + mlist[6] / b) + mlist[8] / 2

      f = [x_min, y_min, x_max, y_max]

      return f



    def forward(self, predictions, target):

        predictions = torch.reshape(predictions, (-1, self.S, self.S, (5 * self.B) + self.C))
        all_loss = 0

        for batch in range(predictions.shape[0]):
            loss1 = 0
            loss2 = 0
            loss3 = 0
            loss4 = 0
            loss5 = 0
            for i in range(self.S):
              for j in range(self.S):
                if intersection_over_union(intersection_over_union_small1(predictions, i, j, self.S), intersection_over_union_small1(target, i, j, self.S)) > intersection_over_union(intersection_over_union_small2(predictions, i, j, self.S), intersection_over_union_small2(target, i, j, self.S)):
                  needed_box = predictions[0:5]
                  nonimp_box = predictions[5:10]
                else:
                  needed_box = predictions[5:10]
                  nonimp_box = predictions[0:5]
                  
                loss1 += (needed_box[0] - target[0]) ** 2 + (needed_box[1] - target[1]) ** 2

                loss2 += (math.sqrt(needed_box[2]) - math.sqrt(target[2])) ** 2 + (math.sqrt(needed_box[3]) - math.sqrt(target[3])) ** 2
                

                loss3 += (needed_box[4] - intersection_over_union(needed_box[:4], target[:4])) ** 2

                loss4 += nonimp_box[4]**2

                loss5 += (predictions[10]-box_target[10])**2+(predictions[11]-target[11])**2+(predictions[12]-target[12])**2
            
                if target[10] != 0 or target[11] != 0 or target[12] != 0:
                  gigaloss = self.lambda_coord * loss1 + self.lambda_coord * loss2 + loss3 + self.lambda_noobj * loss4 + loss5
                else:
                  gigaloss = self.lambda_noobj * loss4
         
         
         return gigaloss

        pass

        #С новым годом